In [1]:
import pandas as pd

In [2]:
def read_csv_files(destination, dates, result=None):
    if result is None:
        dfs = []
        for date in dates:
            filename = f"..\\web_scrapping\\Bxl_{destination}\\booking_{date}.csv"
            df = pd.read_csv(filename)
            dfs.append(df)
        result = pd.concat(dfs, axis=0, ignore_index=True)
    else:
        for date in dates:
            filename = f"..\\web_scrapping\\Bxl_{destination}\\booking_{date}.csv"
            df = pd.read_csv(filename)
            result = pd.concat([result, df], axis=0, ignore_index=True)
    return result

In [3]:
dates = ["14_may_06_PM", "15_may_07_AM", "15_may_10_AM", "15_may_01_PM", "15_may_04_PM"]
dates.append("15_may_07_PM")
dates.append("16_may_07_AM")
dates.append("16_may_10_AM")
dates.append("16_may_01_PM")
dates.append("16_may_04_PM")
dates.append("16_may_07_PM")
dates.append("17_may_07_AM")
dates.append("17_may_10_AM")

In [4]:
df1 = read_csv_files( "Jakarta", dates)

In [5]:
dates_2 = ["17_may_02_PM"]
df4 = read_csv_files( "Jakarta", dates_2)

In [6]:
def transfomed_df(df=df1):
    df['airline_company'] = df['airline_company'].str.split(",").str[0]
    df['out_stop_num'] = df['out_stop_num'].str.split(' ').str[0]
    df['in_stop_num'] = df['in_stop_num'].str.split(' ').str[0]

    split_df = df.pop('price_ticket').str.rsplit(' ', n=1, expand=True).rename(columns={0: 'ticket_price', 1: 'currency'})
    df = df.join(split_df)
    df['ticket_price'] = df['ticket_price'].str.replace(',', '.').str.replace(' ', '').astype(float)

    date_cols = [col for col in df.columns if col.endswith('_date')]
    year= '2023'
    for col in date_cols:
        df[col] = pd.to_datetime(df[col] + ' ' + year, format='%b %d %Y')
    
    time_cols = [col for col in df.columns if col.endswith('_time')]
    df[time_cols] = df[time_cols].apply(lambda x: pd.to_datetime(x, format='%I:%M %p').dt.strftime('%H:%M'))

    duration_cols = [col for col in df.columns if col.endswith('_duration')]
    df[duration_cols] = df[duration_cols].applymap(lambda x: pd.to_timedelta(x.replace('h', ' hours ').replace('m', ' min')))

    return df

In [7]:
def transfomed_df_2(df=df1):
    air_cols = [col for col in df.columns if col.endswith('_airline_company')]
    # Make sure the values in the selected columns are strings
    df[air_cols] = df[air_cols].astype(str)

    # Split the values in the selected columns on a comma and keep only the first element
    df[air_cols] = df[air_cols].apply(lambda x: x.str.split(",").str[0])
    
    df['out_stop_num'] = df['out_stop_num'].str.split(' ').str[0]
    df['in_stop_num'] = df['in_stop_num'].str.split(' ').str[0]

    split_df = df.pop('price_ticket').str.rsplit(' ', n=1, expand=True).rename(columns={0: 'ticket_price', 1: 'currency'})
    df = df.join(split_df)
    df['ticket_price'] = df['ticket_price'].str.replace(',', '.').str.replace(' ', '').astype(float)

    date_cols = [col for col in df.columns if col.endswith('_date')]
    year= '2023'
    for col in date_cols:
        df[col] = pd.to_datetime(df[col] + ' ' + year, format='%b %d %Y')
    
    time_cols = [col for col in df.columns if col.endswith('_time')]
    df[time_cols] = df[time_cols].apply(lambda x: pd.to_datetime(x, format='%I:%M %p').dt.strftime('%H:%M'))

    duration_cols = [col for col in df.columns if col.endswith('_duration')]
    df[duration_cols] = df[duration_cols].applymap(lambda x: pd.to_timedelta(x.replace('h', ' hours ').replace('m', ' min')))

    return df


In [8]:
Jakarta_cleaned_data = transfomed_df()

In [9]:
Jakarta_cleaned_data_2 = transfomed_df_2(df4)
Jakarta_cleaned_data_2

,out_airline_company,in_airline_company,dep_city,arr_city,out_dep_date,out_dep_time,out_duration,out_stop_num,out_arr_date,out_arr_time,in_dep_date,in_dep_time,in_duration,in_stop_num,in_arr_date,in_arr_time,hour_scrap,day_scrap,ticket_price,currency
0,Lufthansa,Qatar Airways,BRU,CGK,2023-08-01,09:05,0 days 17:35:00,1,2023-08-02,07:40,2023-08-15,09:05,1 days 10:05:00,1,2023-08-16,14:10,14,17,1707.94,€
1,Qatar Airways,Lufthansa,BRU,CGK,2023-08-01,09:05,0 days 17:35:00,1,2023-08-02,07:40,2023-08-15,19:00,0 days 18:10:00,2,2023-08-16,08:10,14,17,2335.91,€
2,Qatar Airways,Qatar Airways,BRU,CGK,2023-08-01,09:05,1 days 07:25:00,1,2023-08-02,21:30,2023-08-15,09:05,1 days 10:05:00,1,2023-08-16,14:10,14,17,1711.24,€
3,Lufthansa,Lufthansa,BRU,CGK,2023-08-01,20:05,0 days 17:00:00,2,2023-08-02,18:05,2023-08-15,19:00,0 days 18:10:00,2,2023-08-16,08:10,14,17,2472.61,€
4,Lufthansa,Lufthansa,BRU,CGK,2023-08-01,20:05,0 days 17:00:00,2,2023-08-02,18:05,2023-08-15,19:00,0 days 18:35:00,2,2023-08-16,08:35,14,17,2489.70,€
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Lufthansa,Lufthansa,BRU,CGK,2023-08-01,20:05,0 days 21:35:00,2,2023-08-02,22:40,2023-08-15,19:00,0 days 20:35:00,2,2023-08-16,10:35,14,17,2708.94,€
94,Lufthansa,Lufthansa,BRU,CGK,2023-08-01,14:40,0 days 22:25:00,2,2023-08-02,18:05,2023-08-15,19:00,0 days 20:20:00,2,2023-08-16,10:20,14,17,2691.85,€
95,Lufthansa,Lufthansa,BRU,CGK,2023-08-01,19:15,0 days 22:25:00,2,2023-08-02,22:40,2023-08-15,19:00,0 days 20:20:00,2,2023-08-16,10:20,14,17,2472.61,€
96,Lufthansa,Lufthansa,BRU,CGK,2023-08-01,13:45,0 days 23:20:00,2,2023-08-02,18:05,2023-08-15,19:00,0 days 23:55:00,2,2023-08-16,13:55,14,17,2502.71,€


In [10]:
# Create a boolean mask to identify rows with different outbound and inbound airlines
mask = Jakarta_cleaned_data_2['out_airline_company'] != Jakarta_cleaned_data_2['in_airline_company']

# Use the mask to get the rows with different outbound and inbound airlines
result = Jakarta_cleaned_data_2[mask]

result

,out_airline_company,in_airline_company,dep_city,arr_city,out_dep_date,out_dep_time,out_duration,out_stop_num,out_arr_date,out_arr_time,in_dep_date,in_dep_time,in_duration,in_stop_num,in_arr_date,in_arr_time,hour_scrap,day_scrap,ticket_price,currency
0,Lufthansa,Qatar Airways,BRU,CGK,2023-08-01,09:05,0 days 17:35:00,1,2023-08-02,07:40,2023-08-15,09:05,1 days 10:05:00,1,2023-08-16,14:10,14,17,1707.94,€
1,Qatar Airways,Lufthansa,BRU,CGK,2023-08-01,09:05,0 days 17:35:00,1,2023-08-02,07:40,2023-08-15,19:00,0 days 18:10:00,2,2023-08-16,08:10,14,17,2335.91,€
15,Qatar Airways,Lufthansa,BRU,CGK,2023-08-01,09:05,0 days 17:35:00,1,2023-08-02,07:40,2023-08-15,19:00,0 days 18:10:00,2,2023-08-16,08:10,14,17,2335.91,€


Two dataframes, df2 and df3, are merged. df2 includes all csv files with Jakarta dictionaries. The code has been adjusted to only retrieve Tokyo dictionaries, which are available in all files after df2.

In [11]:
dates = ["14_may_06_PM", "15_may_07_AM"]
df2 = read_csv_files( "Tokyo", dates)

In [12]:
df2 = df2.loc[df2['arr_city'] != 'CGK']
df2 = df2.reset_index(drop=True)

In [13]:
dates = ["15_may_10_AM", "15_may_01_PM", "15_may_04_PM", "15_may_07_PM"]
dates.append("16_may_07_AM")
dates.append("16_may_10_AM")
dates.append("16_may_01_PM")
dates.append("16_may_04_PM")
dates.append("16_may_07_PM")
dates.append("17_may_07_AM")
dates.append("17_may_10_AM")

In [14]:
df3 = read_csv_files( "Tokyo", dates)
df2 = pd.concat([df2, df3], ignore_index=True)

In [15]:
Tokyo_cleaned_data=transfomed_df(df2)

In [16]:
dates_3 = ["17_may_02_PM"]
df5 = read_csv_files( "Tokyo", dates_3)

In [17]:
df5 = transfomed_df_2(df5)
dif = df5['out_airline_company'] != df5['in_airline_company']
result = df5[dif]
result

,out_airline_company,in_airline_company,dep_city,arr_city,out_dep_date,out_dep_time,out_duration,out_stop_num,out_arr_date,out_arr_time,in_dep_date,in_dep_time,in_duration,in_stop_num,in_arr_date,in_arr_time,hour_scrap,day_scrap,ticket_price,currency
